In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
%matplotlib widget
import numpy as np
import k3d
import matplotlib.pyplot as plt
from functools import reduce
from raytracing import *
from ipywidgets import *
from matplotlib.animation import FuncAnimation
import matplotlib.animation as ani

matplotlib.animation.FFMpegWriter

In [21]:
def SixMirror(dx=27.77, dy=8.0, dz=16.685, d=4.750, dzF=1.5825, Rfast=25.0, eps=np.zeros((6,5))):

    p5 = np.array([0,0,0])
    p6 = np.array([dx, dy, dzF])
    p1 = np.array([0, dy, dzF])
    p2 = np.array([dx, 2*dy, 0])
    p3 = np.array([d, dy+d, dz])
    p4 = np.array([dx-d, dy-d, dz])
    
    ps = np.stack([p1,p2,p3,p4,p5,p6], axis=0)
    geom = geometry(ps)
    ns = geom['refl']
    ps = geom['mir']
    #add perturbations
    ps += eps[:,:3]
    axx = geom['xin']
    axy = 0.5*(geom['yin']+geom['yout'])
    ns = [rot_vec(rot_vec(ns[i], axx[i], np.deg2rad(eps[i,3])), axy[i], np.deg2rad(eps[i,4])) for i, n in enumerate(ns)]
    
    hi = 12.7
    qi=7.75
    #reference plane is expected between first and last element!
    elements = [CurvedMirror(p=ps[0], n=ns[0], diam=qi, R=Rfast, curv='CC'),\
                Mirror(p=ps[1], n=ns[1], diam=qi),\
                Mirror(p=ps[2], n=ns[2], diam=hi),\
                Mirror(p=ps[3], n=ns[3], diam=hi),\
                Mirror(p=ps[4], n=ns[4], diam=qi),\
                CurvedMirror(p=ps[5], n=ns[5], diam=qi, R=Rfast, curv='CC')]
    return elements, geom

In [22]:
elements, geom = SixMirror()

x0 = 0.5*(elements[0].p + elements[-1].p)
n0 = norm(elements[0].p - elements[-1].p)
rs = ray_bundle(p=x0, n=n0, n_radii=5, n_angles=2, R=0.2)

screen = Screen(p=x0, n=-n0, diam=7.75)
elements.append(screen)

In [5]:
#r0 = x0[None,:] + np.array([[0,0.2,0], [0,0,0.2], [0,0.2,0.2], [0,-0.2,-0.2]])
#s0 = np.broadcast_to(n0, r0.shape)
r0, s0 = x0, n0
ray0 = np.stack([np.atleast_2d(r0), np.atleast_2d(s0)], axis=0)
reig, allseq, alltols = find_eigenray_mpe_debug(elements, ray0, Niter=20)

Finished in 0 steps, reached tol 3.553e-15


In [6]:
traj_eig = propagate_system(elements, reig, Nrt=1)[:,0,:,:] #only positions

In [7]:
traj_eig.shape

(8, 1, 3)

In [8]:
#visualize perturbed mode
#visualize perturbed mode
which=0
ax=2
amp = 1e-1
dists, trajs_pert, modelmats = perturb_one_anim(SixMirror, screen, reig, traj_eig, which=which, ax=ax, rng=np.linspace(-amp, amp, 30))

C:\ProgramData\Anaconda3\lib\site-packages\traittypes\traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [9]:
trajs_pert_ts = traj_to_timeseries(trajs_pert, step=1)
modelmats_ts = modelmats_to_timeseries(modelmats, step=1)

In [ ]:
plot = k3d.plot(camera_auto_fit=True, antialias=True)

for i, el in enumerate(elements[:-1]):
    pel = el.plot(opacity=0.4)
    if i==which:
        pel.color=0xff0000
    pel.model_matrix = modelmats_ts[i]
    plot += pel

#plot_trajs(traj, plot)
for ts in trajs_pert_ts:
    plot += k3d.line(ts, shader='mesh', width=0.2, color=0x00ff00)

plot.display()

In [ ]:
plot.start_auto_play()

In [ ]:
plot.stop_auto_play()

In [ ]:
#Poincare hit pattern

In [10]:
lrs = np.zeros_like(ray0)
lrs[0,...] = 1e-3 #spatial learning rate
lrs[1,...] = 1e-1 #slope learning rate

In [11]:
elements, _ = SixMirror(dx=27.76)

x0 = 0.5*(elements[0].p + elements[-1].p)
n0 = norm(elements[0].p - elements[-1].p)

screen = Screen(p=x0, n=-n0, diam=7.75)
elements.append(screen)

# 27.8, 0.2r0, -2e-2 s0 10 iter
#27.77 0.1 -2e-3 10

r0 = x0[None,:] + np.array([0,0.02,0])
s0 = n0[None,:] + np.array([0,-2e-4,0])
#s0 = np.broadcast_to(n0, r0.shape)

ray0 = np.stack([np.atleast_2d(r0), np.atleast_2d(s0)], axis=0)

In [12]:
reig3, _, tols = find_eigenray_mpe_debug(elements, ray0, Nrt=3, lr=0.01, Niter=50, Nmpe=4)

Finished in 49 steps, reached tol 8.526e-03
Finished in 49 steps, reached tol 7.674e-04
Finished in 49 steps, reached tol 7.121e-04
Finished in 49 steps, reached tol 4.239e-04


In [13]:
plt.figure()
plt.semilogy(tols)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
#traj_hit = propagate_system(elements, reig3, Nrt=2000)[:,0,:,:] #only positions
#hit_scr = traj_hit[7::7,0,:]
traj_hit = propagate_system_at(elements, reig3, which=6, Nrt=1000)[:,0,:,:] #only positions
hit_scr = traj_hit[:,0,:]
hit_idx = np.arange(hit_scr.shape[0])

plt.figure()
line = plt.scatter(hit_scr[:,1], hit_scr[:,2], c=hit_idx, cmap='jet')
plt.title('Poincare hit pattern (screen)')
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')
plt.colorbar()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
fig, ax = plt.subplots()
line = plt.scatter(hit_scr[:,1], hit_scr[:,2], c=hit_idx, cmap='jet')
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')

@interact(dl=(-0.1,0.1,1e-3), drx=(0,0.05,1e-3),dry=(0,0.05,1e-3),dsx=(-0.1,0.1,1e-3),dsy=(-0.1,0.1,1e-3), Nrt=(0,1000,100))
def update(dl, drx=0., dry=0., dsx=0., dsy=0., maxiter=500, Nrt=1000, **kwargs):
        elements, _ = SixMirror(dx=27.77+dl, **kwargs)

        x0 = 0.5*(elements[0].p + elements[-1].p)
        n0 = norm(elements[0].p - elements[-1].p)
        screen = Screen(p=x0, n=-n0, diam=7.75)
        elements.append(screen)
        r0 = x0[None,:] + np.array([0,drx,dry])
        s0 = n0[None,:] + np.array([0,dsx,dsy])
        ray0 = np.stack([np.atleast_2d(r0), np.atleast_2d(s0)], axis=0)
        reig3 = find_eigenray_mpe(elements, ray0, Nrt=3, lr=0.01, Niter=50, Nmpe=4)
        traj_hit = propagate_system_at(elements, reig3, which=6, Nrt=Nrt)[:,0,:,:] #only positions
        hit_scr = traj_hit[:,0,:]
        hit_idx = np.arange(hit_scr.shape[0])
  
        line.set_offsets(hit_scr[:,1:])
        line.set_array(hit_idx)
        ax.relim()
        ax.autoscale_view()
        fig.canvas.draw_idle()

In [30]:
FFmpeg = ani.writers['ffmpeg'](fps=30, bitrate=12e3)

In [28]:
1/30.

0.03333333333333333

In [31]:
fig, ax = plt.subplots()
line = plt.scatter(hit_scr[:,1], hit_scr[:,2], c=hit_idx, cmap='jet')
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')

def init():
    line.set_offsets([[], []])
    line.set_array([])
    return line,

def animate(i, drx=0., dry=0., dsx=0.01, dsy=0., maxiter=500, Nrt=1000):
    anipar = np.linspace(-0.15, 0.15, 400)
    dl = anipar[i]
    elements, _ = SixMirror(dx=27.77+dl)
    x0 = 0.5*(elements[0].p + elements[-1].p)
    n0 = norm(elements[0].p - elements[-1].p)
    screen = Screen(p=x0, n=-n0, diam=7.75)
    elements.append(screen)
    r0 = x0[None,:] + np.array([0,drx,dry])
    s0 = n0[None,:] + np.array([0,dsx,dsy])
    ray0 = np.stack([np.atleast_2d(r0), np.atleast_2d(s0)], axis=0)
    reig3 = find_eigenray_mpe(elements, ray0, Nrt=3, lr=0.01, Niter=50, Nmpe=4)
    traj_hit = propagate_system_at(elements, reig3, which=6, Nrt=Nrt)[:,0,:,:] #only positions
    hit_scr = traj_hit[:,0,:]
    hit_idx = np.arange(hit_scr.shape[0])

    line.set_offsets(hit_scr[:,1:])
    line.set_array(hit_idx)
    #ax.relim()
    #ax.autoscale_view()
    return line,
anim = FuncAnimation(fig, animate, init_func=init, frames=400, interval=30, blit=True)
anim.save('hits2.mp4', writer=FFmpeg)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\lukas\Box\Lukas\03_Theory\ABCD\Raytracing\raytracing.py:83: RuntimeWarning: invalid value encountered in greater
  msk = np.abs(sn)>np.finfo(np.float32).eps
C:\Users\lukas\Box\Lukas\03_Theory\ABCD\Raytracing\raytracing.py:90: RuntimeWarning: invalid value encountered in greater
  x[(d>self.r),:] = np.nan


ValueError: need at least one array to stack

In [ ]:
anim = FuncAnimation(fig, animate, init_func=init, frames=200, interval=20, blit=True)

In [ ]:
init()

In [ ]:
animate(0)